In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

### import graffiti complaints 2013-2016

In [2]:
g = pd.read_csv('311_graffiti_2013_2016.csv')
g.head(2)

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,24703192,01/02/2013 12:42:41 PM,01/23/2013 02:31:06 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Graffiti or Vandalism,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24708485,01/02/2013 01:51:20 PM,NaN,DSNY,Department of Sanitation,Graffiti,Graffiti,NaN,11213.0,304 UTICA AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.667577,-73.931248,"(40.66757653265371, -73.93124757101009)"


In [3]:
len(g)

60934

In [4]:
g = g.dropna(subset=['X Coordinate (State Plane)', 'Y Coordinate (State Plane)'])

In [5]:
g = g[['Unique Key', 'Created Date', 'X Coordinate (State Plane)', 'Y Coordinate (State Plane)']]

In [6]:
g.isnull().sum()

Unique Key                    0
Created Date                  0
X Coordinate (State Plane)    0
Y Coordinate (State Plane)    0
dtype: int64

In [7]:
g['Created Date'] = pd.to_datetime(g['Created Date'])
g['created_year'] = g['Created Date'].dt.year
g.head(2)

,Unique Key,Created Date,X Coordinate (State Plane),Y Coordinate (State Plane),created_year
1,24708485,2013-01-02 13:51:20,1003323.0,182500.0,2013
2,24708486,2013-01-02 13:50:55,1022237.0,261331.0,2013


In [8]:
g['created_year'].unique()

array([2013, 2014, 2015, 2016])

In [9]:
g['created_year'].dtypes

dtype('int64')

In [10]:
g = g[g['created_year']!=2016]

In [11]:
g.head()

,Unique Key,Created Date,X Coordinate (State Plane),Y Coordinate (State Plane),created_year
1,24708485,2013-01-02 13:51:20,1003323.0,182500.0,2013
2,24708486,2013-01-02 13:50:55,1022237.0,261331.0,2013
3,24713583,2013-01-03 18:50:27,995145.0,186535.0,2013
4,24701077,2013-01-02 00:15:04,1038525.0,197029.0,2013
5,24702407,2013-01-02 13:51:01,1024179.0,264588.0,2013


In [12]:
g.reset_index(inplace=True)
g.head(2)

,index,Unique Key,Created Date,X Coordinate (State Plane),Y Coordinate (State Plane),created_year
0,1,24708485,2013-01-02 13:51:20,1003323.0,182500.0,2013
1,2,24708486,2013-01-02 13:50:55,1022237.0,261331.0,2013


In [13]:
geometry = [Point(xy) for xy in zip(g['X Coordinate (State Plane)'], g['Y Coordinate (State Plane)'])]
#df = df.drop(['Lon', 'Lat'], axis=1)
crs = {'init': 'epsg:2263'}
g_geo = gpd.GeoDataFrame(g, crs=crs, geometry=geometry)

In [14]:
g_geo = g_geo.drop('index',axis=1)

In [15]:
type(g_geo)

geopandas.geodataframe.GeoDataFrame

In [16]:
g_geo = g_geo.to_crs({'init':'epsg:4269'})

### import census block groups

In [17]:
cbg = gpd.read_file('cb_2015_36_bg_500k.shp')

In [18]:
cbg.head(2)

,AFFGEOID,ALAND,AWATER,BLKGRPCE,COUNTYFP,GEOID,LSAD,NAME,STATEFP,TRACTCE,geometry
0,1500000US360610211000,0,307945,0,061,360610211000,BG,0,36,021100,"POLYGON ((-73.968082 40.8207, -73.967982575439..."
1,1500000US360290131014,557271,351509,4,029,360290131014,BG,4,36,013101,POLYGON ((-78.89711856256349 42.75125713600959...


In [19]:
cbg = cbg[['AFFGEOID', 'GEOID', 'COUNTYFP', 'geometry']]

In [20]:
cbg['COUNTYFP'] = cbg['COUNTYFP'].astype(str)

In [21]:
#061 NYC, 081 QN, 085 SI, 047 BK, 005 BX
cbg = cbg.loc[cbg['COUNTYFP'].isin(['061', '081', '085', '047', '005'])]

In [22]:
len(cbg)

6442

In [23]:
cbg = cbg.to_crs({'init':'epsg:4269'})

In [24]:
print cbg.crs
print g_geo.crs

{'init': 'epsg:4269'}
{'init': 'epsg:4269'}


In [25]:
cbg.isnull().sum()

AFFGEOID    0
GEOID       0
COUNTYFP    0
geometry    0
dtype: int64

In [26]:
g_geo.head()

,Unique Key,Created Date,X Coordinate (State Plane),Y Coordinate (State Plane),created_year,geometry
0,24708485,2013-01-02 13:51:20,1003323.0,182500.0,2013,POINT (-73.9312475710101 40.6675764931536)
1,24708486,2013-01-02 13:50:55,1022237.0,261331.0,2013,POINT (-73.86262272666154 40.88388553463825)
2,24713583,2013-01-03 18:50:27,995145.0,186535.0,2013,POINT (-73.96072027397858 40.67866550190102)
3,24701077,2013-01-02 00:15:04,1038525.0,197029.0,2013,POINT (-73.80423819099072 40.70730934852464)
4,24702407,2013-01-02 13:51:01,1024179.0,264588.0,2013,POINT (-73.85558021542973 40.89281638929326)


### merge census block groups and graffiti complaints

In [27]:
graffiti = gpd.sjoin(cbg, g_geo, how='left')

In [28]:
print len(graffiti)
print len(cbg)

44129
6442


In [29]:
graffiti.isnull().sum()

AFFGEOID                         0
GEOID                            0
COUNTYFP                         0
geometry                         0
index_right                   1524
Unique Key                    1524
Created Date                  1524
X Coordinate (State Plane)    1524
Y Coordinate (State Plane)    1524
created_year                  1524
dtype: int64

In [30]:
import numpy as np

In [31]:
graffiti = graffiti.replace(np.nan, 0)

In [32]:
graffiti['created_year'] = graffiti['created_year'].astype(int)

In [33]:
graffiti['created_year'] = graffiti['created_year'].replace(2015, 1)
graffiti['created_year'] = graffiti['created_year'].replace(2014, 1)
graffiti['created_year'] = graffiti['created_year'].replace(2013, 1)
graffiti['graffiti_count'] = graffiti['created_year']

### sum the values in 'graffiti_count' to count the number of graffiti complaints between 2013-2015

In [34]:
graf_counts = pd.DataFrame(graffiti['graffiti_count'].groupby(graffiti['AFFGEOID']).sum())

In [35]:
graf_counts = graf_counts.reset_index()

In [36]:
graf_counts.head()

,AFFGEOID,graffiti_count
0,1500000US360050001000,0
1,1500000US360050001001,0
2,1500000US360050002000,0
3,1500000US360050002001,5
4,1500000US360050002002,4


In [38]:
graffiti['AFFGEOID'] = graffiti['AFFGEOID'].astype(str)

In [39]:
#graffiti[graffiti['GEOID']==360050002000]

In [40]:
print min(graf_counts['graffiti_count'])
print max(graf_counts['graffiti_count'])

0
110


In [41]:
#print len(graf_counts['GEOID'].unique())
#print len(graffiti['GEOID'].unique())

### import precincts

In [42]:
p = gpd.read_file('Police Precincts/geo_export_00b06dd5-82ff-48a1-9401-f80ee795bc70.shp')

In [43]:
p.head(2)

,geometry,precinct,shape_area,shape_leng
0,(POLYGON ((-74.0438776157395 40.69018767637665...,1.0,4.718213e+07,79979.389271
1,POLYGON ((-73.98863862848766 40.72293372026369...,5.0,1.808880e+07,18676.124259


In [44]:
p.crs

{'init': u'epsg:4326'}

In [45]:
p = p.to_crs({'init':'epsg:4269'})

In [46]:
graffiti2 = gpd.sjoin(p, g_geo, how='left')

In [47]:
graffiti2.head(2)

,geometry,precinct,shape_area,shape_leng,index_right,Unique Key,Created Date,X Coordinate (State Plane),Y Coordinate (State Plane),created_year
0,(POLYGON ((-74.0438776157395 40.69018767637665...,1.0,4.718213e+07,79979.389271,8876,26407143,2013-09-30 18:50:08,984171.0,197253.0,2013
0,(POLYGON ((-74.0438776157395 40.69018767637665...,1.0,4.718213e+07,79979.389271,23821,30487558,2015-04-26 11:22:11,979547.0,197606.0,2015


In [48]:
len(graffiti2)

42548

In [49]:
graffiti2['created_year'] = graffiti2['created_year'].replace(2015, 1)
graffiti2['created_year'] = graffiti2['created_year'].replace(2014, 1)
graffiti2['created_year'] = graffiti2['created_year'].replace(2013, 1)
graffiti2['graffiti_count'] = graffiti2['created_year']

In [50]:
graf_counts2 = pd.DataFrame(graffiti2['graffiti_count'].groupby(graffiti2['precinct']).sum())

In [51]:
graf_counts2.reset_index(inplace=True)

In [52]:
graf_counts2.head()

,precinct,graffiti_count
0,1.0,358
1,5.0,1550
2,6.0,247
3,7.0,834
4,9.0,786


In [53]:
len(graffiti2[graffiti2['precinct']==1.0])

358

In [54]:
graf_counts2['precinct'] = graf_counts2['precinct'].astype(int)

In [55]:
pct = pd.read_csv('170418_3_pp_w_foursquare_prep.csv')
pct = pct.drop('Unnamed: 0',axis=1)
pct.head(2)

,Precinct,meanfemale,meanelderl,meanwhite_,meanblack_,meanasian_,meanhh,meanhh_chi,meanhh_liv,meanedu_hi,...,ratio_com,ratio_res,ratio_off,ratio_ret,ratio_gar,ratio_strg,ratio_fac,meandiv_ar,sumFreq,sumbars
0,1,0.49825,0.07057,0.74474,0.0329,0.15403,245.66000,0.43539,0.44691,0.76004,...,0.36046,0.29212,0.18941,0.04615,0.0096,0.02441,0.00128,0.61482,210,84
1,5,0.48870,0.11206,0.37708,0.0523,0.47612,144.87719,0.25946,0.42496,0.43872,...,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.66470,130,40


In [56]:
pct = pd.merge(pct, graf_counts2, left_on='Precinct', right_on='precinct')

In [57]:
pct = pct.drop('precinct',axis=1)

In [58]:
pct.head()

,Precinct,meanfemale,meanelderl,meanwhite_,meanblack_,meanasian_,meanhh,meanhh_chi,meanhh_liv,meanedu_hi,...,ratio_res,ratio_off,ratio_ret,ratio_gar,ratio_strg,ratio_fac,meandiv_ar,sumFreq,sumbars,graffiti_count
0,1,0.49825,0.07057,0.74474,0.03290,0.15403,245.66000,0.43539,0.44691,0.76004,...,0.29212,0.18941,0.04615,0.00960,0.02441,0.00128,0.61482,210,84,358
1,5,0.48870,0.11206,0.37708,0.05230,0.47612,144.87719,0.25946,0.42496,0.43872,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.66470,130,40,1550
2,6,0.49669,0.09824,0.83960,0.02427,0.08845,108.34375,0.38256,0.56843,0.77366,...,0.51035,0.09272,0.06752,0.02762,0.01078,0.00135,0.55937,385,112,247
3,7,0.51967,0.14202,0.37639,0.09229,0.34302,178.68750,0.18567,0.44041,0.36402,...,0.57893,0.01635,0.04601,0.02086,0.00748,0.00162,0.46920,197,52,834
4,9,0.51897,0.07346,0.63156,0.07417,0.15358,135.56716,0.20519,0.50965,0.59596,...,0.57351,0.04600,0.06289,0.01275,0.00675,0.00034,0.52823,424,143,786


In [59]:
pct.to_csv('pct_data_4_18.csv')

### import noise data

In [62]:
noise = pd.read_csv('ny_cb_shp_w_noise_counts')

In [63]:
noise.head(2)

,Unnamed: 0,AFFGEOID,GEOID,geometry,Descriptor
0,0,1500000US360850156031,360850156031,"POLYGON ((-74.16134900000002 40.529374, -74.15...",30
1,1,1500000US360050004001,360050004001,"POLYGON ((-73.85870299999999 40.815405, -73.85...",38


In [64]:
noise = noise.drop('Unnamed: 0',axis=1)

In [65]:
noise['AFFGEOID'].dtypes

dtype('int64')

In [67]:
graf_counts['AFFGEOID'].dtypes

dtype('O')

In [68]:
graf_counts['AFFGEOID'] = graf_counts['AFFGEOID'].astype(str)

In [69]:
noise['AFFGEOID'] = noise['AFFGEOID'].astype(str)

In [124]:
noise_graf = pd.merge(noise, graf_counts, on='AFFGEOID')

In [125]:
noise.columns

Index([u'AFFGEOID', u'GEOID', u'geometry', u'Descriptor'], dtype='object')

In [126]:
graf_counts.columns

Index([u'AFFGEOID', u'graffiti_count'], dtype='object')

In [127]:
noise.head(2)

,AFFGEOID,GEOID,geometry,Descriptor
0,1500000US360850156031,360850156031,"POLYGON ((-74.16134900000002 40.529374, -74.15...",30
1,1500000US360050004001,360050004001,"POLYGON ((-73.85870299999999 40.815405, -73.85...",38


In [128]:
graf_counts.head(2)

,AFFGEOID,graffiti_count
0,1500000US360050001000,0
1,1500000US360050001001,0


In [129]:
print len(cbg['AFFGEOID'].unique())
print len(noise['AFFGEOID'].unique())
print len(graf_counts['AFFGEOID'].unique())
print len(noise_graf['AFFGEOID'].unique())

6442
6304
6442
6296


In [130]:
noise_graf.head()

,AFFGEOID,GEOID,geometry,Descriptor,graffiti_count
0,1500000US360850156031,360850156031,"POLYGON ((-74.16134900000002 40.529374, -74.15...",30,0
1,1500000US360050004001,360050004001,"POLYGON ((-73.85870299999999 40.815405, -73.85...",38,16
2,1500000US360811447001,360811447001,"POLYGON ((-73.77740299999999 40.758214, -73.77...",36,2
3,1500000US360810269013,360810269013,"POLYGON ((-73.88256200000001 40.747663, -73.87...",76,1
4,1500000US360470444001,360470444001,"POLYGON ((-73.969112 40.61701, -73.96774000000...",9,2


In [131]:
noise_graf = noise_graf.rename(columns={"Descriptor":"noise_count"})

In [132]:
noise_graf.head(2)

,AFFGEOID,GEOID,geometry,noise_count,graffiti_count
0,1500000US360850156031,360850156031,"POLYGON ((-74.16134900000002 40.529374, -74.15...",30,0
1,1500000US360050004001,360050004001,"POLYGON ((-73.85870299999999 40.815405, -73.85...",38,16


In [133]:
noise_graf['AFFGEOID'] = noise_graf['AFFGEOID'].astype(str)

In [134]:
noise_graf = noise_graf.drop('geometry',axis=1)

In [135]:
noise_graf.to_csv('noise_graf_counts_cbg.csv')

### variables from martin

In [136]:
ind = pd.read_csv('170418_3_cbg_w_foursquare_prep.csv')

In [137]:
ind.head(2)

,Unnamed: 0,Id2,AFFGEOID_x,GEOID_x,pop,female_r,elderly_r,white_r,black_r,asian_r,...,ratio_com,ratio_res,ratio_off,ratio_ret,ratio_gar,ratio_strg,ratio_fac,div_areas,sumFreq,sumbars
0,1,360610211000,1500000US360610211000,360610211000,0,0.0000,0.00000,0.00000,0.0,0.00000,...,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0,0
1,2,360470228003,1500000US360470228003,360470228003,816,0.4277,0.00613,0.96324,0.0,0.03676,...,0.4488,0.10344,0.03381,0.05086,0.02349,0.01204,0.29418,0.6958,0,0


In [138]:
ind = ind.drop('Unnamed: 0',axis=1)

In [139]:
ind.head(2)

,Id2,AFFGEOID_x,GEOID_x,pop,female_r,elderly_r,white_r,black_r,asian_r,hh,...,ratio_com,ratio_res,ratio_off,ratio_ret,ratio_gar,ratio_strg,ratio_fac,div_areas,sumFreq,sumbars
0,360610211000,1500000US360610211000,360610211000,0,0.0000,0.00000,0.00000,0.0,0.00000,0,...,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0,0
1,360470228003,1500000US360470228003,360470228003,816,0.4277,0.00613,0.96324,0.0,0.03676,485,...,0.4488,0.10344,0.03381,0.05086,0.02349,0.01204,0.29418,0.6958,0,0


In [140]:
ind.columns

Index([u'Id2', u'AFFGEOID_x', u'GEOID_x', u'pop', u'female_r', u'elderly_r',
       u'white_r', u'black_r', u'asian_r', u'hh', u'hh_child_6', u'hh_living_',
       u'edu_high_r', u'non_eng_r', u'unemployed', u'vacancy_r', u'med_rent',
       u'med_income', u'merried_r', u'longercomm', u'BldgArea', u'ComArea',
       u'ResArea', u'OfficeArea', u'RetailArea', u'GarageArea', u'StrgeArea',
       u'FactryArea', u'OtherArea', u'NumBldgs', u'sum_areas', u'ratio_com',
       u'ratio_res', u'ratio_off', u'ratio_ret', u'ratio_gar', u'ratio_strg',
       u'ratio_fac', u'div_areas', u'sumFreq', u'sumbars'],
      dtype='object')

In [141]:
ind = ind.rename(columns={'AFFGEOID_x':'AFFGEOID', 'GEOID_x':'GEOID'})

In [142]:
len(ind)

6442

In [143]:
features = pd.merge(ind, noise_graf, on='AFFGEOID')

In [144]:
features.head(2)

,Id2,AFFGEOID,GEOID_x,pop,female_r,elderly_r,white_r,black_r,asian_r,hh,...,ratio_ret,ratio_gar,ratio_strg,ratio_fac,div_areas,sumFreq,sumbars,GEOID_y,noise_count,graffiti_count
0,360470228003,1500000US360470228003,360470228003,816,0.42770,0.00613,0.96324,0.00000,0.03676,485,...,0.05086,0.02349,0.01204,0.29418,0.69580,0,0,360470228003,17,9
1,360470231003,1500000US360470231003,360470231003,722,0.53047,0.04294,0.43629,0.44044,0.00000,62,...,0.00155,0.00000,0.00000,0.00000,0.48441,0,0,360470231003,368,6


In [145]:
features = features.drop('GEOID_y',axis=1)
features = features.rename(columns={'GEOID_x':'GEOID'})
features.head(2)

,Id2,AFFGEOID,GEOID,pop,female_r,elderly_r,white_r,black_r,asian_r,hh,...,ratio_off,ratio_ret,ratio_gar,ratio_strg,ratio_fac,div_areas,sumFreq,sumbars,noise_count,graffiti_count
0,360470228003,1500000US360470228003,360470228003,816,0.42770,0.00613,0.96324,0.00000,0.03676,485,...,0.03381,0.05086,0.02349,0.01204,0.29418,0.69580,0,0,17,9
1,360470231003,1500000US360470231003,360470231003,722,0.53047,0.04294,0.43629,0.44044,0.00000,62,...,0.00000,0.00155,0.00000,0.00000,0.00000,0.48441,0,0,368,6


In [146]:
features['graf_r'] = features['graffiti_count']/features['pop']
features['noise_r'] = features['noise_count']/features['pop']

In [147]:
features.head(2)

,Id2,AFFGEOID,GEOID,pop,female_r,elderly_r,white_r,black_r,asian_r,hh,...,ratio_gar,ratio_strg,ratio_fac,div_areas,sumFreq,sumbars,noise_count,graffiti_count,graf_r,noise_r
0,360470228003,1500000US360470228003,360470228003,816,0.42770,0.00613,0.96324,0.00000,0.03676,485,...,0.02349,0.01204,0.29418,0.69580,0,0,17,9,0.011029,0.020833
1,360470231003,1500000US360470231003,360470231003,722,0.53047,0.04294,0.43629,0.44044,0.00000,62,...,0.00000,0.00000,0.00000,0.48441,0,0,368,6,0.008310,0.509695


In [148]:
len(features)

6296

In [149]:
print len(ind['AFFGEOID'].unique())
print len(noise_graf['AFFGEOID'].unique())
print len(noise_graf)

6442
6296
6296


In [150]:
features.to_csv('cbg_features_4_19.csv')

In [152]:
len(features.columns)

45